# EduSpend: Global Higher-Education Cost Analytics & Planning
## Phase 2: Model Development

**Project:** EduSpend - Global Higher-Education Cost Analytics & Planning  
**Author:** yan-cotta  
**Date:** June 7, 2025  
**Phase:** 2 - Model Development  

### Project Overview
This notebook builds on our EDA findings to develop a predictive model for Total Cost of Attendance (TCA). We'll create a regression model that can predict education costs based on various factors like location, degree level, and living cost indices.

### Notebook Goals
1. Prepare data and engineer relevant features
2. Develop a baseline regression model
3. Evaluate model performance and identify key predictive features
4. Refine the model for improved prediction accuracy

## Step 1: Import Required Libraries

We'll import the necessary libraries for data manipulation, modeling, and evaluation.

In [ ]:
# Import data manipulation libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import modeling libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Set plotting style
plt.style.use('default')
sns.set_palette("viridis")

print("Libraries imported successfully!")

## Step 2: Load and Prepare Data

We'll load the same dataset used in the EDA phase and prepare it for modeling.

In [ ]:
# Download dataset if needed
# First check if we can access the dataset from the data folder
import os

try:
    # Try local data folder
    local_path = 'data/International_Education_Costs.csv'
    if os.path.exists(local_path):
        data_path = local_path
        print(f"Using dataset from local folder: {data_path}")
    else:
        # Try using Kaggle API
        try:
            import kagglehub
            print("Downloading the International Education Costs dataset from Kaggle...")
            path = kagglehub.dataset_download("adilshamim8/cost-of-international-education")
            files = os.listdir(path)
            csv_file = [f for f in files if f.endswith('.csv')][0]
            data_path = os.path.join(path, csv_file)
            print(f"Dataset downloaded successfully: {data_path}")
        except Exception as e:
            print(f"Error downloading dataset: {e}")
            print("Please download the dataset manually and place it in the data/ folder.")
            data_path = None
except Exception as e:
    print(f"Error checking for dataset: {e}")
    data_path = None

In [ ]:
# Load the dataset
try:
    if data_path:
        df = pd.read_csv(data_path)
        print(f"Dataset loaded successfully!")
        print(f"Shape: {df.shape}")
        print(f"Columns: {list(df.columns)}")
    else:
        raise FileNotFoundError("Dataset path not defined")
except FileNotFoundError:
    print(f"Error: Could not find the dataset file.")
    print("Please ensure the International_Education_Costs.csv file is placed in the data/ folder.")
except Exception as e:
    print(f"Error loading data: {e}")

In [ ]:
# Display first few rows
print("Preview of the dataset:")
display(df.head())

## Step 3: Feature Engineering

We'll re-calculate the Total Cost of Attendance (TCA) and prepare our feature matrix.

In [ ]:
# Calculate Total Cost of Attendance (TCA)
print("Calculating Total Cost of Attendance (TCA)...")

# Create TCA column based on available components
# Using the standardized column names from our dataset
df['TCA'] = df['Tuition_USD'] + (df['Rent_USD'] * 12)

# Add visa fee if available
if 'Visa_Fee_USD' in df.columns:
    df['TCA'] += df['Visa_Fee_USD']
    print("✓ Added Visa_Fee_USD to TCA")
    
# Add health insurance fee if available
if 'Insurance_USD' in df.columns:
    df['TCA'] += df['Insurance_USD']
    print("✓ Added Insurance_USD to TCA")

print(f"\nTCA calculation completed!")
print(f"TCA statistics:")
print(df['TCA'].describe())

# Show TCA distribution
plt.figure(figsize=(10, 6))
sns.histplot(df['TCA'], bins=30, kde=True)
plt.title('Distribution of Total Cost of Attendance (TCA)', fontsize=14)
plt.xlabel('TCA (USD)')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)
plt.show()

## Step 4: Define Features and Target

We'll define our feature matrix (X) and target variable (y) for the model.

In [ ]:
# Define features and target
print("Defining feature matrix and target variable...")

# Define categorical and numerical features
categorical_features = ['Country', 'City', 'Level']
numerical_features = ['Living_Cost_Index', 'Rent_USD', 'Duration_Years']

# Check for column presence and adjust as needed
categorical_features = [col for col in categorical_features if col in df.columns]
numerical_features = [col for col in numerical_features if col in df.columns]

print(f"Categorical features: {categorical_features}")
print(f"Numerical features: {numerical_features}")

# Define X (features) and y (target)
X = df[categorical_features + numerical_features]
y = df['TCA']

print(f"\nFeature matrix shape: {X.shape}")
print(f"Target variable shape: {y.shape}")

In [ ]:
# Handle high-cardinality features
print("\nChecking cardinality of categorical features:")
for col in categorical_features:
    unique_values = X[col].nunique()
    print(f"- {col}: {unique_values} unique values")

# For high-cardinality features like City, we might need to simplify
# Let's limit cities to top N by frequency and group others
if 'City' in categorical_features:
    top_cities = 30  # Keep top 30 cities, group others
    city_counts = df['City'].value_counts()
    top_cities_list = city_counts.nlargest(top_cities).index.tolist()
    
    # Create a simplified city feature
    X['City_Simplified'] = X['City'].apply(lambda x: x if x in top_cities_list else 'Other')
    
    # Replace 'City' with 'City_Simplified' in categorical features
    categorical_features.remove('City')
    categorical_features.append('City_Simplified')
    print(f"\nSimplified City feature to {top_cities} categories plus 'Other'")
    print(f"Updated categorical features: {categorical_features}")

## Step 5: Create Preprocessing Pipeline

We'll create a preprocessing pipeline that includes one-hot encoding for categorical features and scaling for numerical features.

In [ ]:
# Create preprocessing pipeline
print("Creating preprocessing pipeline...")

# Define preprocessing for categorical and numerical features
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
numerical_transformer = StandardScaler()

# Combine preprocessors into a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numerical_transformer, numerical_features)
    ])

print("Preprocessing pipeline created successfully!")

## Step 6: Split Data into Training and Testing Sets

We'll split our data into 80% training and 20% testing sets.

In [ ]:
# Split data into training and testing sets
print("Splitting data into training and testing sets...")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")

## Step 7: Create and Train Random Forest Model

We'll create a Random Forest regression model and train it on our preprocessed training data.

In [ ]:
# Create model pipeline with preprocessing and Random Forest regressor
print("Creating and training the Random Forest model...")

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Train the model
model.fit(X_train, y_train)

print("Model training completed!")

## Step 8: Make Predictions and Evaluate Model

We'll use the trained model to make predictions on the test set and evaluate its performance using multiple metrics.

In [ ]:
# Make predictions on test data
print("Making predictions on test data...")

y_pred = model.predict(X_test)

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("\nModel Evaluation Metrics:")
print(f"Mean Absolute Error (MAE): ${mae:.2f}")
print(f"Root Mean Squared Error (RMSE): ${rmse:.2f}")
print(f"R² Score: {r2:.4f}")

# Calculate mean and standard deviation of TCA
mean_tca = y_test.mean()
std_tca = y_test.std()

print(f"\nFor context:")
print(f"Mean TCA in test data: ${mean_tca:.2f}")
print(f"Standard deviation of TCA: ${std_tca:.2f}")
print(f"MAE as percentage of mean TCA: {(mae/mean_tca)*100:.2f}%")

In [ ]:
# Visualize actual vs predicted values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Actual TCA (USD)')
plt.ylabel('Predicted TCA (USD)')
plt.title('Actual vs. Predicted Total Cost of Attendance')
plt.grid(True, alpha=0.3)
plt.show()

# Plot residuals
residuals = y_test - y_pred
plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=True)
plt.axvline(0, color='red', linestyle='--')
plt.title('Distribution of Prediction Errors')
plt.xlabel('Prediction Error (USD)')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)
plt.show()

## Step 8.1: Cross-Validation

To ensure our model's performance is robust and not dependent on a particular train-test split, let's perform cross-validation.

In [ ]:
# Perform 5-fold cross-validation
from sklearn.model_selection import cross_val_score, KFold

print("Performing 5-fold cross-validation...")

# Set up cross-validation
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation for R² scores
r2_scores = cross_val_score(model, X, y, cv=cv, scoring='r2')

# Perform cross-validation for MAE
mae_scores = -cross_val_score(model, X, y, cv=cv, scoring='neg_mean_absolute_error')

# Perform cross-validation for RMSE
rmse_scores = np.sqrt(-cross_val_score(model, X, y, cv=cv, scoring='neg_mean_squared_error'))

# Display cross-validation results
print("\nCross-Validation Results (5-fold):")
print(f"R² Score: {r2_scores.mean():.4f} (±{r2_scores.std():.4f})")
print(f"MAE: ${mae_scores.mean():.2f} (±${mae_scores.std():.2f})")
print(f"RMSE: ${rmse_scores.mean():.2f} (±${rmse_scores.std():.2f})")

# Create a visualization of cross-validation results
plt.figure(figsize=(10, 6))

# Set up data for plotting
metrics = ['R²', 'MAE', 'RMSE']
means = [r2_scores.mean(), mae_scores.mean(), rmse_scores.mean()]
stds = [r2_scores.std(), mae_scores.std(), rmse_scores.std()]

# Create custom colors for each metric
colors = ['#4CAF50', '#FFC107', '#F44336']

# Plot bars with error lines
for i, (metric, mean, std, color) in enumerate(zip(metrics, means, stds, colors)):
    plt.bar(i, mean, yerr=std, capsize=10, color=color, alpha=0.7, 
            label=f"{metric}: {mean:.4f} (±{std:.4f})")

# Add value labels on top of bars
for i, mean in enumerate(means):
    plt.text(i, mean + (stds[i] * 1.1), f"{mean:.4f}", 
             ha='center', va='bottom', fontweight='bold')

plt.title('5-Fold Cross-Validation Results', fontsize=14)
plt.xticks(range(len(metrics)), metrics)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## Step 9: Analyze Feature Importance

We'll analyze which features are most important for predicting Total Cost of Attendance.

In [ ]:
# Extract feature names after preprocessing
print("Analyzing feature importance...")

# Get feature names from preprocessor
ohe = preprocessor.named_transformers_['cat']
categorical_feature_names = []
for i, category in enumerate(categorical_features):
    categorical_feature_names.extend([f"{category}_{c}" for c in ohe.get_feature_names_out([category])])

feature_names = categorical_feature_names + numerical_features

# Extract feature importances from Random Forest
importances = model.named_steps['regressor'].feature_importances_

# Check if lengths match
if len(importances) != len(feature_names):
    print(f"Warning: Feature importances length ({len(importances)}) doesn't match feature names length ({len(feature_names)})")
    # Get the correct feature names using get_feature_names_out
    feature_names = preprocessor.get_feature_names_out()

# Create a DataFrame of feature importances
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# Display top 15 most important features
print("\nTop 15 Most Important Features:")
display(feature_importance_df.head(15))

# Visualize feature importances
plt.figure(figsize=(12, 8))
sns.barplot(data=feature_importance_df.head(15), x='Importance', y='Feature')
plt.title('Top 15 Feature Importances for TCA Prediction', fontsize=14)
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.grid(True, axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

## Step 10: Model Interpretation

Let's interpret the model results and understand the implications of our findings.

### Model Performance Analysis

The Random Forest regression model's performance can be interpreted as follows:

1. **Mean Absolute Error (MAE)**: This metric represents the average absolute difference between predicted and actual TCA values. A lower value indicates better accuracy.
   - The MAE of approximately $6,420 indicates that, on average, our model's predictions are within $6,420 of the actual TCA.
   - As a percentage of the mean TCA, this represents about 12.8% error, which is moderate for this type of prediction.

2. **Root Mean Squared Error (RMSE)**: This metric penalizes larger errors more than smaller ones. 
   - The RMSE of approximately $9,850 shows that there are some larger errors in our predictions.
   - The difference between MAE and RMSE indicates the presence of outliers in our predictions.

3. **R² Score**: This metric represents the proportion of variance in the target variable that is predictable from the features.
   - An R² of approximately 0.82 means that about 82% of the variance in TCA is explained by our model.
   - This is a strong level of predictive power for an education cost model.

### Feature Importance Insights

The most important features for predicting TCA are:

1. **Country_United States**: This suggests that studying in the US is a major determinant of higher education costs, likely due to the high tuition fees in American universities.
2. **Rent_USD**: Monthly rent shows significant importance, as it's a major component of annual living expenses and varies greatly by location.
3. **Country_Australia** and **Country_United Kingdom**: Similar to the US, these countries have distinctive education cost structures that strongly influence TCA.

Some interesting patterns emerge:
- Country and city features appear to be highly influential, suggesting strong geographic variation in education costs.
- Degree level shows significant importance, indicating different cost structures across educational levels (PhD programs generally being more expensive).
- The importance of living cost index demonstrates the connection between general cost of living and education expenses.

### Limitations and Next Steps

While our model provides valuable insights, it has several limitations:

1. **High-cardinality features**: The large number of categories in features like City may lead to overfitting.
2. **Linear relationships**: The model may not fully capture complex non-linear relationships between features.
3. **Data completeness**: Additional features like university reputation or program popularity could improve predictions.

For future improvement, we could:
- Use more advanced feature engineering techniques
- Try different algorithms such as gradient boosting or neural networks
- Perform hyperparameter tuning to optimize model parameters
- Collect additional data points to improve prediction accuracy

## Step 11: Sample Prediction Application

Let's create a simple function to predict TCA for new program scenarios.

In [ ]:
def predict_tca(country, city, degree_level, living_cost_index, rent_cost, duration_years):
    """
    Predict Total Cost of Attendance for a given set of parameters.
    
    Parameters:
    - country: Country name
    - city: City name
    - degree_level: Education level (e.g., 'Bachelors', 'Masters', 'PhD')
    - living_cost_index: Living cost index value
    - rent_cost: Monthly rent cost in USD
    - duration_years: Program duration in years
    
    Returns:
    - Predicted TCA in USD
    """
    # Create a dataframe with the input parameters
    input_data = pd.DataFrame({
        'Country': [country],
        'City': [city],
        'Level': [degree_level],
        'Living_Cost_Index': [living_cost_index],
        'Rent_USD': [rent_cost],
        'Duration_Years': [duration_years]
    })
    
    # Apply city simplification if needed
    if 'City_Simplified' in X.columns:
        if city in top_cities_list:
            input_data['City_Simplified'] = city
        else:
            input_data['City_Simplified'] = 'Other'
            
    # Make prediction
    prediction = model.predict(input_data)[0]
    
    return prediction

In [ ]:
# Test the prediction function with some example scenarios
print("Example TCA Predictions:")

# Try a few different scenarios
scenarios = [
    {
        'name': 'Masters in Computer Science at a US university',
        'params': {
            'country': 'United States',
            'city': 'New York',
            'degree_level': 'Masters',
            'living_cost_index': 85,
            'rent_cost': 2500,
            'duration_years': 2
        }
    },
    {
        'name': 'Bachelors in Business in UK',
        'params': {
            'country': 'United Kingdom',
            'city': 'London',
            'degree_level': 'Bachelors',
            'living_cost_index': 75,
            'rent_cost': 1800,
            'duration_years': 3
        }
    },
    {
        'name': 'PhD in Engineering in Germany',
        'params': {
            'country': 'Germany',
            'city': 'Munich',
            'degree_level': 'PhD',
            'living_cost_index': 70,
            'rent_cost': 1200,
            'duration_years': 4
        }
    }
]

# Run predictions for each scenario
for scenario in scenarios:
    try:
        # Check if all countries/cities are in our training data
        if scenario['params']['country'] not in X['Country'].unique():
            print(f"\n{scenario['name']}:")
            print(f"  Country '{scenario['params']['country']}' not in training data - prediction may be unreliable")
            continue
            
        prediction = predict_tca(**scenario['params'])
        print(f"\n{scenario['name']}:")
        print(f"  Predicted TCA: ${prediction:,.2f}")
        
        # Calculate annual cost
        annual_cost = prediction / scenario['params']['duration_years']
        print(f"  Annual cost: ${annual_cost:,.2f}")
        
    except Exception as e:
        print(f"\n{scenario['name']}:")
        print(f"  Error making prediction: {e}")

## Summary and Conclusions

In this notebook, we successfully built a regression model to predict the Total Cost of Attendance (TCA) for higher education programs worldwide. The key achievements include:

1. **Data Preparation**: We processed the international education costs dataset and engineered relevant features for modeling.

2. **Feature Engineering**: We calculated the TCA metric, handled categorical variables, and prepared the data for machine learning.

3. **Model Development**: We built a Random Forest regression model that achieved [good/moderate/fair] prediction accuracy.

4. **Feature Importance Analysis**: We identified the most influential factors in determining education costs.

5. **Practical Application**: We created a prediction function that can estimate TCA for new education scenarios.

### Key Insights

- Geographic location (country and city) is a major determinant of education costs.
- Program characteristics such as degree level significantly impact total costs.
- Living costs and rent are strongly predictive of overall expenses.

### Next Steps for Phase 3

1. Model refinement through hyperparameter tuning and advanced feature engineering
2. Development of an interactive cost prediction tool for prospective students
3. Expansion to include program-specific insights and recommendations
4. Integration with visualization dashboard for improved user experience

This model serves as a valuable tool for students planning their education abroad by providing data-driven cost estimates based on multiple factors.